✅ Базовый парсинг локального проекта. Просто помещаем этот юпитер-ноут в корень своего проекта и нажимаем выполнить. На выходе получим JSON кода всех файлов проекта с учетом иерархии вложенности, готовый для подачи в контекст любой нейросети без необходимости копирования кода всех файлов проекта.

In [4]:
import os
import json
from pathlib import Path

def read_text_file(path: Path) -> str:
    """
    Пробует прочитать текстовый файл с поддержкой английских и русских кодировок.
    Порядок приоритета: UTF-8 (с/без BOM) → Windows Latin-1 → Windows Cyrillic.
    """
    encodings = [
        "utf-8",
        "utf-8-sig",     # BOM
        "utf-16",
        "cp1251",        # Русский Windows
        "cp1252",        # Западный Windows
        "latin1",        # Fallback
        "iso-8859-1",
        "cp866",         # DOS Cyrillic
        "koi8-r",        # Unix Cyrillic
    ]
    for enc in encodings:
        try:
            with open(path, "r", encoding=enc) as f:
                return f.read()
        except (UnicodeDecodeError, UnicodeError):
            continue
    return f"<Ошибка: не удалось прочитать файл '{path}' ни в одной из поддерживаемых кодировок>"

def project_to_json(root_dir=".", extensions=(".py", ".yaml", ".yml", ".json", ".md", ".txt", ".ipynb")):
    project = {}
    root = Path(root_dir).resolve()
    for path in root.rglob("*"):
        if path.is_file() and path.suffix in extensions:
            rel_path = str(path.relative_to(root))
            try:
                if path.suffix == ".ipynb":
                    # Обработка Jupyter Notebook
                    with open(path, "r", encoding="utf-8") as f:
                        nb = json.load(f)
                    cells_content = []
                    for cell in nb.get("cells", []):
                        if cell["cell_type"] in ("code", "markdown"):
                            source = "".join(cell["source"])
                            cells_content.append(f"# [{cell['cell_type']}]")
                            cells_content.append(source)
                    project[rel_path] = "\n".join(cells_content)
                else:
                    # Обычные текстовые файлы
                    project[rel_path] = read_text_file(path)
            except Exception as e:
                project[rel_path] = f"<Ошибка при чтении: {e}>"
    return json.dumps(project, indent=2, ensure_ascii=False)

if __name__ == "__main__":
    output = project_to_json()
    with open("project_as_json.json", "w", encoding="utf-8") as f:
        f.write(output)
    print("✅ Проект успешно сохранён в project_as_json.json")

✅ Проект успешно сохранён в project_as_json.json
